In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
from sklearn.datasets import fetch_california_housing
import pandas as pd
california_housing = fetch_california_housing(as_frame=True)

In [ ]:
dev = qml.device("default.qubit", wires=4)




def statepreparation(f):
    qml.AmplitudeEmbedding(features=f, wires=range(3), normalize=True)
    qml.BasisState(np.array([0]), wires=3)


def layer(W):
    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
    qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)
    qml.Rot(W[2, 0], W[2, 1], W[2, 2], wires=2)
    qml.Rot(W[3, 0], W[3, 1], W[3, 2], wires=3)
    for i in range(3):
        for l in range(i+1,4):
            qml.CNOT(wires=[i, l])

@qml.qnode(dev)
def circuit(weights, x): 
    statepreparation(x) 
    for W in weights:
        layer(W) 
    return qml.expval(qml.PauliZ(4))

def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2 
    loss = loss / len(labels)
    return loss

def accuracy(labels, predictions): 
    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)
    return loss


def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [ ]:
num_qubits = 4
num_layers = 6

weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)
df = pd.DataFrame(data=california_housing.data, columns=california_housing.feature_names)
inputs = df.values.tolist()  
targets = list(california_housing.target) 